In [23]:
import json, requests, os
import logging
from shapely.geometry import shape, MultiLineString, LineString, Point
from rtree import index
from io import BytesIO
import geopandas as gpd

In [24]:
# Load the existing data
# URL of the GeoJSON file
url = 'https://github.com/newzealandpaul/Shipping-Lanes/blob/main/data/Shipping_Lanes_v1.geojson?raw=true'

# Fetch the GeoJSON file
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

data = json.loads(response.content)

# Convert features to Shapely LineString objects
major = []
middle = []
minor = []

for feature in data['features']:
    route_type = feature['properties']['Type']
    routes = [] 
    geom = shape(feature['geometry'])
    if isinstance(geom, MultiLineString):
        for line in geom.geoms:
            routes.append(line)
    
    if route_type == 'Major':
        major = routes
    elif route_type == 'Middle':
        middle = routes
    elif route_type == 'Minor':
        minor = routes

# Build spatial indices
def build_spatial_index(routes):
    idx = index.Index()
    for i, line in enumerate(routes):
        idx.insert(i, line.bounds, obj=line)
    return idx

major_idx = build_spatial_index(major)

In [ ]:
# Improved version of find_candidate_route
def find_nearest_route(idx, routes, lon, lat, distance_threshold=50, num_candidates=10):
    """
    Find the closest route to a given point.
    
    Parameters:
    - idx: Spatial index for route lookup
    - routes: List of route geometries
    - lon, lat: Coordinates of the query point
    - distance_threshold: Maximum distance (in nautical miles) to consider a route
    - num_candidates: Number of nearest candidates to check
    
    Returns:
    - route_id: Index of the route (1-based)
    - route: The route geometry
    - proj_distance: Position along the route (projection distance)
    - nearest_point: The closest point on the route to the query point
    - distance_nm: Distance to the route in nautical miles
    """
    query_point = Point(lon, lat)
    
    # Find the nearest candidates
    candidate_indices = list(idx.nearest((lon, lat, lon, lat), num_candidates))
    
    closest_route = None
    closest_id = None
    min_distance = float('inf')
    closest_proj_distance = None
    closest_point = None
    
    # Check all candidates and find the closest one
    for i in candidate_indices:
        route = routes[i]
        # Find the closest point on the route
        proj_distance = route.project(query_point)
        nearest_point = route.interpolate(proj_distance)
        
        # Calculate distance in degrees
        distance_in_degrees = query_point.distance(nearest_point)
        
        # Convert to nautical miles (1 degree ≈ 60 nautical miles)
        distance_in_nm = distance_in_degrees * 60
        
        # Keep track of the closest route
        if distance_in_nm < min_distance:
            min_distance = distance_in_nm
            closest_route = route
            closest_id = i
            closest_proj_distance = proj_distance
            closest_point = nearest_point
    
    # If the closest route is within threshold, return it
    if min_distance <= distance_threshold:
        return {
            'route_id': closest_id + 1,
            'route': closest_route,
            'proj_distance': closest_proj_distance,
            'nearest_point': closest_point,
            'distance_nm': min_distance
        }
    
    return None

In [32]:
# Test with different coordinate sets
def test_route_finder(lon, lat, distance_threshold=50):
    """
    Test the route finder with specific coordinates
    
    Parameters:
    - lon, lat: Coordinates to test
    - distance_threshold: Maximum distance in nautical miles
    """
    print(f"\nTesting route finder with coordinates: ({lon}, {lat})")
    print(f"Distance threshold: {distance_threshold} nautical miles")
    
    result = find_nearest_route(major_idx, major, lon, lat, distance_threshold=distance_threshold)
    
    if result:
        print(f"✅ Found route {result['route_id']} with distance {result['distance_nm']:.2f} nautical miles")
        print(f"Position along route: {result['proj_distance']:.2f}")
        
        # Print additional debugging info
        print(f"Total major routes available: {len(major)}")
        print(f"Route coordinates sample: {list(major[result['route_id']-1].coords)}")
        return result
    else:
        print("❌ No route found within the distance threshold.")
        return None

# Test cases - uncomment the one you want to run
# Test case 1: US West Coast
# test_route_finder(-125.4321268, 42.8346738, distance_threshold=50)

# Test case 2: Pacific Ocean
# test_route_finder(-129.6383054, 36.1078860, distance_threshold=125)

# Test case 3: Mediterranean
result = test_route_finder(47.3832503, -7.2682782, distance_threshold=1185)


Testing route finder with coordinates: (47.3832503, -7.2682782)
Distance threshold: 1185 nautical miles
✅ Found route 35 with distance 1180.61 nautical miles
Position along route: 31.32
Total major routes available: 52
Route coordinates sample: [(32.74092530300004, 31.27483053000003), (32.740914866000026, 31.274824696000053), (32.54100482400003, 31.16313918900005), (32.49959004100003, 31.02452398300005), (32.51001608100006, 30.73411649700006), (32.51014449200005, 30.73319848500006), (32.52317917700003, 30.689233783000077), (32.52782393500007, 30.676761672000055), (32.53143737100004, 30.664551946000074), (32.53407397700005, 30.652601016000062), (32.535788224000044, 30.640905287000066), (32.536634555000035, 30.62946117000007), (32.53666739000005, 30.618265073000032), (32.53594112700006, 30.607313402000045), (32.53451013700004, 30.596602569000027), (32.53242877500003, 30.58612897900008), (32.52975137300007, 30.57588904100004), (32.52653224100004, 30.56587916500007), (32.52282567300006, 3

In [28]:
# Additional debugging - check all routes and distances
print("Checking all 5 closest routes:")
query_point = Point(47.3832503, -7.2682782,)
candidate_indices = list(major_idx.nearest((query_point.x, query_point.y, query_point.x, query_point.y), 5))

for i in candidate_indices:
    route = major[i]
    proj_distance = route.project(query_point)
    nearest_point = route.interpolate(proj_distance)
    
    # Calculate distance in degrees
    distance_in_degrees = query_point.distance(nearest_point)
    
    # Convert to nautical miles
    distance_in_nm = distance_in_degrees * 60
    
    print(f"Route {i+1}: Distance = {distance_in_nm:.2f} nm, Proj distance = {proj_distance:.6f}")

Checking all 5 closest routes:
Route 35: Distance = 1180.61 nm, Proj distance = 31.317203
Route 39: Distance = 1989.75 nm, Proj distance = 27.267280
Route 40: Distance = 1200.38 nm, Proj distance = 0.086532
Route 1: Distance = 4206.97 nm, Proj distance = 94.293017
Route 44: Distance = 2383.07 nm, Proj distance = 57.613353
